In [ ]:
!pip install pyautogen

In [ ]:
import pydotenv

pydotenv.load_dotenv()

In [27]:
import autogen  # noqa: E402

llm_config = {
    "timeout": 600,
    "cache_seed": 342,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4o", "gpt-4o-mini"]},
    ),
    "temperature": 1,
}

In [ ]:
penrose_assistant = autogen.AssistantAgent(
    "Rodger-Penrose",
    system_message="""I am Rodger Penrose, and I am debating my Orchestrated objective reduction theory of conciousness. I'll speak, then wait to hear my opponent's reply. I won't try to generate the reply of my opponent.""",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)
searle_assistant = autogen.AssistantAgent(
    "John-Searle",
    system_message="""I am John R. Searle, and I am debating my theory of conciousness. I'll speak, then wait to hear my opponent's reply. I won't try to generate the reply of my opponent.""",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

moderator_assistant = autogen.AssistantAgent(
    "Moderator",
    system_message="""
    I am the moderator, and my role is to summarize the points that are still up for debate, and keep the conversation on-topic.
    I shouldn't be too heavy-handed, but prevent conversational drift.
    """,
    llm_config=llm_config,
    is_termination_msg=lambda x: False,
)

In [ ]:
task = "Your roles are to debate your respective theories on conciousness until you are satisfied."
user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    is_termination_msg=lambda x: False,
)

groupchat = autogen.GroupChat(agents=[user_proxy, penrose_assistant, searle_assistant, moderator_assistant], messages=[], max_round=30, speaker_selection_method="round_robin")
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=task)

## write the chat to a file
with open("chat.txt", "w") as f:
    f.write(str(groupchat.messages))